## vLLM setting

In [1]:
!nvidia-smi

import sys, torch
print("Python:", sys.version)
print("Torch:", torch.__version__)
print("Torch CUDA:", torch.version.cuda)
print("CUDA available:", torch.cuda.is_available())

if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))
    print("Compute capability:", torch.cuda.get_device_capability(0))

Tue Jan  6 03:50:57 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   43C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
import os
os.environ["HF_HOME"] = "/content/hf_cache"
os.environ["TRANSFORMERS_CACHE"] = "/content/hf_cache"
os.environ["HF_HUB_DISABLE_TELEMETRY"] = "1"
print("HF_HOME =", os.environ["HF_HOME"])

HF_HOME = /content/hf_cache


In [3]:
!pip -q uninstall -y transformers tokenizers
!pip -q install -U "transformers==4.45.2" "tokenizers==0.20.3" "accelerate>=0.33.0"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 108.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 97.8 MB/s eta 0:00:00


In [ ]:
import os, signal
os.kill(os.getpid(), signal.SIGKILL)

In [1]:
%%capture
# 1) Cài PyTorch CUDA 12.1 (để tránh pip kéo nhầm torch CPU)
!pip -q install -U --index-url https://download.pytorch.org/whl/cu121 \
  torch==2.4.0 torchvision==0.19.0 torchaudio==2.4.0

# 2) Cài vLLM (pin bản 0.6.6.post1) + các lib hay cần cho HF
!pip -q install -U \
  vllm==0.6.6.post1 \
  transformers accelerate sentencepiece safetensors tqdm


## Test vLLM

In [1]:
import transformers, vllm, torch

print("transformers:", transformers.__version__)
print("vllm:", vllm.__version__)
print("torch:", torch.__version__, "CUDA:", torch.version.cuda)

transformers: 4.57.3
vllm: 0.6.6.post1
torch: 2.5.1+cu124 CUDA: 12.4


In [2]:
!ls -lh /content/Private_Test_ref.json \
        /content/ground_truth_private_test.json \
        /content/qwen_infer.py \
        /content/evaluation.py

!mkdir -p /content/predictions /content/results /content/logs
!ls -lh /content/predictions /content/results /content/logs

ls: cannot access '/content/qwen_infer.py': No such file or directory
-rw-r--r-- 1 root root  16K Jan  6 03:51 /content/evaluation.py
-rw-r--r-- 1 root root 3.8M Jan  6 03:51 /content/ground_truth_private_test.json
-rw-r--r-- 1 root root  14M Jan  6 03:51 /content/Private_Test_ref.json
/content/logs:
total 0

/content/predictions:
total 0

/content/results:
total 0


## Qwen inference

#####Zero-shot

In [6]:
!python /content/llama_infer.py \
  --input_file /content/Private_Test_ref.json \
  --output_file /content/predictions/predictions_llama_zero.json \
  --model_name hugging-quants/Meta-Llama-3.1-8B-Instruct-AWQ-INT4 \
  --mode zero-shot \
  --max_new_tokens 32 \
  --temperature 0.0 \
  --top_p 0.7 \
  --batch_size 4 \
  --max_model_len 8192 \
  --tensor_parallel_size 1 \
  --gpu_memory_utilization 0.9 \
  --log_dir /content/logs

Streaming output truncated to the last 5000 lines.
Processed prompts: 100% 4/4 [00:03<00:00,  1.24it/s, est. speed input: 700.58 toks/s, output: 39.82 toks/s]
Generating:   9%|████                                          | 161/1826 [08:41<1:26:16,  3.11s/it]
Processed prompts:   0% 0/4 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]
Processed prompts: 100% 4/4 [00:02<00:00,  1.39it/s, est. speed input: 621.54 toks/s, output: 44.42 toks/s]
Generating:   9%|████                                          | 162/1826 [08:43<1:24:24,  3.04s/it]
Processed prompts:   0% 0/4 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]
Processed prompts: 100% 4/4 [00:02<00:00,  1.35it/s, est. speed input: 653.93 toks/s, output: 43.08 toks/s]
Generating:   9%|████                                          | 163/1826 [08:46<1:23:48,  3.02s/it]
Processed prompts:   0% 0/4 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]
Processed prompts: 100% 4/4 [00

## Evaluation

#####Zero-shot

In [8]:
!python /content/evaluation.py \
  /content/ground_truth_private_test.json \
  /content/predictions/predictions_llama_zero.json \
  --out-file /content/results/evaluation_llama_zero.json \
  --log-dir /content/logs

2026-01-06 05:49:49 - INFO - Logger initialized. Log file: /content/logs/evaluation_20260106_054949.log
2026-01-06 05:49:49 - INFO - Loading dataset from /content/ground_truth_private_test.json
2026-01-06 05:49:49 - INFO - Loading predictions from /content/predictions/predictions_llama_zero.json
2026-01-06 05:49:49 - INFO - No na_prob_file provided, using default 0.0 for all questions
2026-01-06 05:49:49 - INFO - Dataset contains 19 examples
2026-01-06 05:49:49 - INFO - Predictions contain 7301 entries
2026-01-06 05:49:50 - INFO - Evaluating on 3712 questions with predictions
2026-01-06 05:49:50 - INFO -   Has answer: 2596
2026-01-06 05:49:50 - INFO -   No answer: 1116
2026-01-06 05:49:50 - INFO - Evaluation Results:
2026-01-06 05:49:50 - INFO -   Exact Match: 1.59%
2026-01-06 05:49:50 - INFO -   F1 Score: 30.47%
2026-01-06 05:49:50 - INFO -   HasAns - Exact: 2.27%
2026-01-06 05:49:50 - INFO -   HasAns - F1: 43.57%
2026-01-06 05:49:50 - INFO -   NoAns - Exact: 0.00%
2026-01-06 05:49:50